# Hourly Forecasting Carbon Intensity using SARIMA #

## Importing modules ##

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pmdarima as pm
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler
import time

# --- 1. Data Loading ---
df = pd.read_csv('data/df_fuel_ckan.csv')
df['DATETIME'] = pd.to_datetime(df['DATETIME'])

# Filter for history and the test year (2025)
df_filtered = df[df['DATETIME'].dt.year < 2025].copy().set_index('DATETIME')
df_2025 = df[df['DATETIME'].dt.year == 2025].copy().set_index('DATETIME')

# Use 30 days for balance between speed and accuracy
data_train = df_filtered['CARBON_INTENSITY'].resample('h').mean().interpolate().tail(24 * 30)
actual_2025 = df_2025['CARBON_INTENSITY'].resample('h').mean().interpolate()

# --- 2. STABILITY FIX: Simpler, more stable exogenous features ---
def create_time_features(index):
    """Create simple time-based features instead of complex Fourier terms"""
    df_feat = pd.DataFrame(index=index)
    df_feat['hour'] = index.hour / 23.0  # Normalize to [0,1]
    df_feat['day_of_week'] = index.dayofweek / 6.0
    return df_feat.values  # Only 2 features for speed

X_train = create_time_features(data_train.index)
X_future = create_time_features(actual_2025.index)

# --- 3. STABILITY FIX: Scale the target variable ---
scaler_y = StandardScaler()
y_train_scaled = scaler_y.fit_transform(data_train.values.reshape(-1, 1)).flatten()

# --- 4. SARIMAX with conservative settings for convergence ---
start_time = time.time()

print("Fitting SARIMAX model with daily seasonality...")
print(f"Training on {len(data_train)} hours of data...")

stepwise_model = pm.auto_arima(
    y=y_train_scaled,
    X=X_train,
    seasonal=True,
    m=24,                      # 24-hour daily cycle
    start_p=0, start_q=0,      # Start from simplest model
    start_P=0, start_Q=0,      # Start without seasonal components
    max_p=2, max_q=2,          # Conservative limits
    max_P=1, max_Q=1,          # Max 1 for seasonal (faster convergence)
    d=None, D=None,            # Auto-detect
    max_d=1, max_D=1,          # Limit to 1
    test='adf',
    seasonal_test='ocsb',
    stepwise=True,
    n_jobs=1,
    suppress_warnings=True,
    error_action='ignore',
    information_criterion='aic',
    trace=True,
    maxiter=100                # More iterations for stability
)

training_time = time.time() - start_time
print(f"\nModel Training Time: {training_time:.2f} seconds")
print(f"Best model: {stepwise_model.order} x {stepwise_model.seasonal_order}")

# --- 5. Forecasting for 2025 ---
forecast_scaled = stepwise_model.predict(
    n_periods=len(actual_2025),
    X=X_future
)

# Inverse transform to original scale
forecast_values = scaler_y.inverse_transform(forecast_scaled.reshape(-1, 1)).flatten()
forecast_series = pd.Series(forecast_values, index=actual_2025.index)

# --- 6. Evaluation & Visualization ---
mae = mean_absolute_error(actual_2025, forecast_series)
rmse = np.sqrt(mean_squared_error(actual_2025, forecast_series))

print(f"\nPerformance Metrics:")
print(f"MAE: {mae:.2f} gCO2/kWh")
print(f"RMSE: {rmse:.2f} gCO2/kWh")

plt.figure(figsize=(18, 8))

# Plot actuals
plt.plot(actual_2025.index, actual_2025, label='Actual 2025', 
         color='green', alpha=0.3, linewidth=1)

# Plot forecast
plt.plot(forecast_series.index, forecast_series, 
         label=f'SARIMAX Forecast {stepwise_model.order}x{stepwise_model.seasonal_order}', 
         color='red', linestyle='--', alpha=0.9, linewidth=1)

plt.title(f'Optimized SARIMAX: Carbon Intensity Forecast 2025\nMAE: {mae:.2f}, RMSE: {rmse:.2f}, Training: {training_time:.1f}s')
plt.xlabel('Date')
plt.ylabel('gCO2/kWh')
plt.legend(loc='upper right')
plt.grid(True, alpha=0.2)

plt.tight_layout()
plt.show()